In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import time

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Introduction

In [ ]:
train_data = pd.read_csv('../input/tabular-playground-series-aug-2021/train.csv')
test_data = pd.read_csv('../input/tabular-playground-series-aug-2021/test.csv')

In [ ]:
train_data.head()

In [ ]:
test_data.head()

In [ ]:
print('train data shape is {}'.format(train_data.shape))
print('test data shape is {}'.format(test_data.shape))

Let's check if the columns of the data_train and the test_data has other different columns. 

In [ ]:
different_collumn = list(set(train_data.columns)-set(test_data.columns))
print(different_collumn)


This shows that the both data has the same columns.

In [ ]:
missing_values = (train_data.isnull().sum())
print(missing_values[missing_values>0])

No missing values

# Machine Learning Model

In [ ]:
from sklearn.linear_model import LinearRegression
from xgboost import XGBRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import cross_val_score
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
X = train_data.drop(['id','loss'],axis=1)
y = train_data['loss']
X.head()

In [ ]:
standard = StandardScaler()
X_train = standard.fit_transform(X)
X_test =  standard.transform(test_data.drop('id',axis=1))



In [ ]:
print(X_train)

In [ ]:
lr = LinearRegression()
xgb = XGBRegressor(n_estimators = 100,random_state = 0)
dt = DecisionTreeRegressor()
gb = GradientBoostingRegressor(random_state = 0)
cat = CatBoostRegressor()
lg = LGBMRegressor()

In [ ]:
def model_result(model):
    start = time.time()
    model.fit(X_train,y)
    mean_result = cross_val_score(model,X_train,y,cv=5,scoring='neg_mean_squared_error')
    stop = time.time()
    print(f"Time spend : {round((stop-start),2)}s")
    return -1*mean_result.mean()

In [ ]:
print("Linear Regression :  {}".format(np.sqrt(model_result(lr))))


In [ ]:
print("Decision Tree : {}".format(np.sqrt(model_result(dt))))


In [ ]:
print("XGBoost  : {}".format(np.sqrt(model_result(xgb))))


In [ ]:
print("CatBoost  : {}".format(np.sqrt(model_result(cat))))


In [ ]:
print("Light : {}".format(np.sqrt(model_result(lg))))

CatBoost performs the best 

In [ ]:
prediction = cat.predict(X_test)

# Submission

In [ ]:
output = pd.DataFrame({'id': test_data.id,
                       'loss': prediction})
output.to_csv('submission.csv', index=False)